# FIFA Rating Prediction Model - Learning Guide

## 📚 What You'll Learn

This notebook teaches you how to build a machine learning model from scratch. By the end, you'll understand:

1. **How to predict continuous values** (regression) - predicting rating changes
2. **XGBoost algorithm** - what it is, how it works, and why it's powerful
3. **Feature engineering** - creating useful inputs from raw data
4. **Model evaluation** - knowing if your model is good or bad
5. **Real-world application** - two practical features for FIFA predictions

## 🎯 Our Two Features

### Feature 1: Auto-Scout
- **Input**: Player name + current season (24/25)
- **Output**: Predicted next season rating (25/26)
- **Use case**: "Will Lamine Yamal improve next season?"

### Feature 2: What-If Simulator
- **Input**: Player name + custom stats for 25/26 season
- **Output**: Predicted 26/27 rating based on those custom stats
- **Use case**: "If Yamal scores 20 goals in 25/26, what will his 26/27 rating be?"

## 🔄 Timeline
```
24/25 (Past) → 25/26 (Current/Predicted) → 26/27 (Future/What-If)
    ↓              ↓                           ↓
  Known        Feature 1                  Feature 2
  Data         Prediction                 Simulation
```

---

Let's start learning!

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns

# Set random seed for reproducibility
np.random.seed(42)

print("✅ Libraries loaded")

## 1. Load and Explore Data

In [ ]:
# Load cleaned data
df = pd.read_csv('../data/clean/fifa_fbref_merged.csv')

print(f"Dataset: {len(df)} rows × {len(df.columns)} columns")
print(f"\nSeasons: {sorted(df['season'].unique())}")
print(f"Players: {df['player'].nunique()}")
print(f"\nPositions: {df['pos'].value_counts().head(10)}")

df.head()

## 2. Create Target Variable: Rating Change

We need to create **next season's rating** for each player-season so we can predict rating growth.

In [ ]:
# Convert season to numeric (1718 → 2017)
df['season_year'] = df['season'].astype(str).str[:2].astype(int) + 2000

# Convert overall to numeric
df['overall'] = pd.to_numeric(df['overall'], errors='coerce')
df['age'] = pd.to_numeric(df['age_fifa'], errors='coerce')

# Sort by player and season
df = df.sort_values(['player', 'season_year'])

# Create next season's rating
df['next_overall'] = df.groupby('player')['overall'].shift(-1)

# Calculate rating change (Δ OVR)
df['rating_change'] = df['next_overall'] - df['overall']

# Remove last season (no next season data)
df_model = df[df['next_overall'].notna()].copy()

print(f"Training data: {len(df_model)} player-seasons with next season data")
print(f"\nRating Change Distribution:")
print(df_model['rating_change'].describe())

# Visualize rating changes
plt.figure(figsize=(10, 5))
plt.hist(df_model['rating_change'], bins=30, edgecolor='black')
plt.xlabel('Rating Change (Δ OVR)')
plt.ylabel('Count')
plt.title('Distribution of Rating Changes')
plt.axvline(0, color='red', linestyle='--', label='No change')
plt.legend()
plt.show()

df_model[['player', 'season', 'age', 'overall', 'next_overall', 'rating_change']].head(10)

## 2b. Create Next Season Stat Columns

Create next season columns for all stats we want to predict (not just overall rating).


In [ ]:
# Create next season columns for all stats we want to predict

# FIFA Face Card Stats (Outfield)
fifa_stats = ['pace', 'shooting', 'passing', 'dribbling', 'defending', 'physic']
for stat in fifa_stats:
    if stat in df_model.columns:
        df_model[f'next_{stat}'] = df_model.groupby('player')[stat].shift(-1)

# GK Stats
gk_stats = ['goalkeeping_diving', 'goalkeeping_handling', 'goalkeeping_kicking', 
            'goalkeeping_positioning', 'goalkeeping_reflexes', 'goalkeeping_speed']
for stat in gk_stats:
    if stat in df_model.columns:
        df_model[f'next_{stat}'] = df_model.groupby('player')[stat].shift(-1)

# Value
if 'value_eur' in df_model.columns:
    df_model['next_value_eur'] = df_model.groupby('player')['value_eur'].shift(-1)

if 'Playing Time_Min' in df_model.columns:
    df_model['next_Playing Time_Min'] = df_model.groupby('player')['Playing Time_Min'].shift(-1)
if 'potential' in df_model.columns:
    df_model['next_potential'] = df_model.groupby('player')['potential'].shift(-1)
# Per 90 Stats (Outfield)
per90_stats = [
    'Per 90 Minutes_Gls',
    'Per 90 Minutes_Ast', 
    'Per 90 Minutes_Tackles_Tkl',
    'Per 90 Minutes_Int',
    'Per 90 Minutes_KP'
]
for stat in per90_stats:
    if stat in df_model.columns:
        df_model[f'next_{stat}'] = df_model.groupby('player')[stat].shift(-1)

# Per 90 Stats (GK)
gk_per90_stats = [
    'Per 90 Minutes_Performance_Saves',
    'Per 90 Minutes_Performance_CS'
]
for stat in gk_per90_stats:
    if stat in df_model.columns:
        df_model[f'next_{stat}'] = df_model.groupby('player')[stat].shift(-1)

print("✅ Created next season columns for:")
print(f"   - {len([s for s in fifa_stats if s in df_model.columns])} FIFA face card stats")
print(f"   - {len([s for s in gk_stats if s in df_model.columns])} GK attributes")
print(f"   - Value_eur")
print(f"   - {len([s for s in per90_stats if s in df_model.columns])} outfield per-90 stats")
print(f"   - {len([s for s in gk_per90_stats if s in df_model.columns])} GK per-90 stats")
print(f"\nTotal new columns: {len([c for c in df_model.columns if c.startswith('next_')])}")


## 3. Identify Goalkeepers vs Outfield Players

Goalkeepers use completely different stats, so we need separate models. We'll create an indicator here but split the datasets AFTER adding lagged features (section 6).

In [ ]:
# Identify goalkeepers (we'll split AFTER creating lagged features)
df_model['is_gk'] = df_model['pos'].str.contains('GK', na=False)

print(f"Goalkeepers: {df_model['is_gk'].sum()}")
print(f"Outfield: {(~df_model['is_gk']).sum()}")
print(f"\n✅ Created is_gk indicator (will split datasets after lagged features)")

## 4. Define Features

### Why Include FIFA Attributes?
FIFA attributes (pace, shooting, passing, etc.) are **crucial features** because:
- They represent EA Sports' current evaluation of the player
- They capture abilities that might not show in stats (e.g., pace, physical strength)
- They likely influence how EA predicts future ratings
- They're already normalized/scaled values

### Outfield Player Features:
- **FIFA Attributes**: Pace, shooting, passing, dribbling, defending, physical
- **Age & Trajectory**: Age, current rating, potential
- **Performance**: Goals/90, Assists/90, xG, xA (all per-90 stats)
- **Playing Time**: Minutes, matches
- **Passing**: Completion%, progressive passes, key passes
- **Shooting**: Shots/90, accuracy, distance
- **Creation**: Shot-creating actions, goal-creating actions
- **Defensive**: Tackles, interceptions, blocks
- **Possession**: Progressive carries, dribbles, cards

### Goalkeeper Features:
- **FIFA GK Attributes**: Diving, handling, kicking, positioning, reflexes, speed
- **Age & Trajectory**: Age, current rating, potential
- **Performance**: Save%, GA/90, clean sheets (all per-90 stats)
- **Playing Time**: Minutes, matches
- **Penalties**: Penalty kicks faced, saved, save%

In [ ]:
# Outfield features
outfield_features = [
    # Age & Rating
    'age_fifa',
    'overall',
    'potential',

    # FIFA Attributes (EA's current evaluation - VERY important!)
    'pace',
    'shooting',
    'passing',
    'dribbling',
    'defending',
    'physic',
    # Note: value_zscore used instead of value_eur (accounts for inflation)
    
    # Playing Time
    'Playing Time_Min',
    'Playing Time_90s',
    'Playing Time_Starts',
    
    # Per 90 Minutes - Goals & Assists (all per-90 stats, not raw counts)
    'Per 90 Minutes_Gls',
    'Per 90 Minutes_Ast',
    'Per 90 Minutes_G+A',
    'Per 90 Minutes_G-PK',
    'Per 90 Minutes_G+A-PK',
    'Per 90 Minutes_xG',
    'Per 90 Minutes_xAG',
    'Per 90 Minutes_xG+xAG',
    'Per 90 Minutes_npxG',
    'Per 90 Minutes_npxG+xAG',
    
    # Per 90 Minutes - Shooting
    'Standard_Sh/90',
    'Standard_SoT/90',
    'Standard_SoT%',
    'Standard_G/Sh',
    'Standard_G/SoT',
    'Standard_Dist',
    'Expected_npxG/Sh',
    
    # Per 90 Minutes - Passing
    'Per 90 Minutes_Total_Cmp',
    'Per 90 Minutes_Total_Att',
    'Total_Cmp%',
    'Per 90 Minutes_Total_TotDist',
    'Per 90 Minutes_Total_PrgDist',
    'Per 90 Minutes_Short_Cmp',
    'Per 90 Minutes_Short_Att',
    'Short_Cmp%',
    'Per 90 Minutes_Medium_Cmp',
    'Per 90 Minutes_Medium_Att',
    'Medium_Cmp%',
    'Per 90 Minutes_Long_Cmp',
    'Per 90 Minutes_Long_Att',
    'Long_Cmp%',
    'Per 90 Minutes_Expected_xA',
    'Per 90 Minutes_Expected_A-xAG',
    'Per 90 Minutes_KP',
    'Per 90 Minutes_1/3',
    'Per 90 Minutes_PPA',
    'Per 90 Minutes_CrsPA',
    
    # Per 90 Minutes - Creation
    'SCA_SCA90',
    'Per 90 Minutes_SCA Types_PassLive',
    'Per 90 Minutes_SCA Types_PassDead',
    'Per 90 Minutes_SCA Types_TO',
    'Per 90 Minutes_SCA Types_Sh',
    'Per 90 Minutes_SCA Types_Fld',
    'Per 90 Minutes_SCA Types_Def',
    'GCA_GCA90',
    'Per 90 Minutes_GCA Types_PassLive',
    'Per 90 Minutes_GCA Types_PassDead',
    'Per 90 Minutes_GCA Types_TO',
    'Per 90 Minutes_GCA Types_Sh',
    'Per 90 Minutes_GCA Types_Fld',
    'Per 90 Minutes_GCA Types_Def',
    
    # Per 90 Minutes - Defensive
    'Per 90 Minutes_Tackles_Tkl',
    'Per 90 Minutes_Tackles_TklW',
    'Per 90 Minutes_Tackles_Def 3rd',
    'Per 90 Minutes_Tackles_Mid 3rd',
    'Per 90 Minutes_Tackles_Att 3rd',
    'Per 90 Minutes_Challenges_Tkl',
    'Per 90 Minutes_Challenges_Att',
    'Challenges_Tkl%',
    'Per 90 Minutes_Challenges_Lost',
    'Per 90 Minutes_Blocks_Blocks',
    'Per 90 Minutes_Blocks_Sh',
    'Per 90 Minutes_Blocks_Pass',
    'Per 90 Minutes_Int',
    'Per 90 Minutes_Tkl+Int',
    'Per 90 Minutes_Clr',
    'Per 90 Minutes_Err',
    
    # Per 90 Minutes - Possession
    'Per 90 Minutes_PKatt',
    'Per 90 Minutes_CrdY',
    'Per 90 Minutes_CrdR',
    'Per 90 Minutes_PrgC',
    'Per 90 Minutes_PrgP',
    'Per 90 Minutes_PrgR',
]

# GK features
gk_features = [
    # Age & Rating
    'age_fifa',
    'overall',
    'potential',
    
    # FIFA Attributes for GK
    'goalkeeping_diving',
    'goalkeeping_handling',
    'goalkeeping_kicking',
    'goalkeeping_positioning',
    'goalkeeping_reflexes',
    'goalkeeping_speed',
    # Note: value_zscore used instead of value_eur (accounts for inflation)
    
    # Playing Time
    'Playing Time_Min',
    'Playing Time_90s',
    
    # GK Performance (per-90 stats)
    'Performance_GA90',
    'Per 90 Minutes_Performance_SoTA',
    'Per 90 Minutes_Performance_Saves',
    'Performance_Save%',
    'Per 90 Minutes_Performance_W',
    'Per 90 Minutes_Performance_D',
    'Per 90 Minutes_Performance_L',
    'Per 90 Minutes_Performance_CS',
    'Performance_CS%',
    'Per 90 Minutes_Penalty Kicks_PKatt',
    'Per 90 Minutes_Penalty Kicks_PKA',
    'Per 90 Minutes_Penalty Kicks_PKsv',
    'Per 90 Minutes_Penalty Kicks_PKm',
    'Penalty Kicks_Save%',
]

print(f"✅ Defined {len(outfield_features)} outfield features and {len(gk_features)} GK features")

## 4b. EXPERIMENT: Simplified Core Features

**Hypothesis**: Fewer, high-signal features might reduce noise and improve generalization.

Instead of 85+ features, focus on ~30-40 core stats that EA likely prioritizes:
- FIFA card attributes (already captures EA's view)
- Age trajectory  
- Playing time
- Goals/Assists (actual + expected)
- Key passing/shooting/defensive stats
- Wage/Value (market perception)

**Benefits of simplification**:
- ✅ Less overfitting risk
- ✅ Faster training
- ✅ More interpretable 
- ✅ Focus on signal, not noise

In [ ]:
# SIMPLIFIED FEATURE SET - Core stats only

# Outfield: ~30 core features
outfield_features_simple = [
    # Age & Rating (4)
    'age_fifa',
    'overall',
    'potential',
    
    # FIFA Attributes - EA's evaluation (6)
    'pace',
    'shooting',
    'passing',
    'dribbling',
    'defending',
    'physic',
    
    # Playing Time (2)
    'Playing Time_Min',
    'Playing Time_90s',
    
    # Goals & Assists - Core attacking output (4)
    'Per 90 Minutes_Gls',
    'Per 90 Minutes_Ast',
    'Per 90 Minutes_G+A',
    'Per 90 Minutes_G-PK',
    
    # Expected Stats - Quality of chances (2)
    'Per 90 Minutes_xG',
    'Per 90 Minutes_xAG',
    
    # Shooting Quality (2)
    'Standard_Sh/90',
    'Standard_SoT%',
    
    # Passing Quality (3)
    'Total_Cmp%',
    'Per 90 Minutes_KP',
    'Per 90 Minutes_PrgP',
    
    # Defensive contribution (2)
    'Per 90 Minutes_Tackles_Tkl',
    'Per 90 Minutes_Int',
    
    # Creation (1)
    'SCA_SCA90',
]

# GK: Keep existing (already lean at 26 features)
gk_features_simple = gk_features.copy()

print(f"✅ Simplified feature sets defined:")
print(f"   Outfield: {len(outfield_features_simple)} features (vs {len(outfield_features)} full)")
print(f"   GK: {len(gk_features_simple)} features")
print(f"\n📊 Reduction: {len(outfield_features) - len(outfield_features_simple)} fewer features")
print(f"   This removes: detailed passing breakdown, creation types, defensive zones, possession stats")

## 5. Add Engineered Features

Create additional features that help predict rating changes:
- **Age squared** (captures non-linear age effects - young players grow fast, old players decline)
- **Career stage** (youth/prime/veteran)
- **Performance vs Rating** (overperforming/underperforming)
- **Wage z-score** (standardized within season to account for wage inflation across FIFA versions)

In [ ]:
def add_engineered_features(df, is_gk=False):
    """
    Add engineered features that help predict rating changes
    """
    df = df.copy()
    
    # Age-related features
    df['age_squared'] = df['age'] ** 2
    
    # Career stage (impacts growth/decline)
    df['is_youth'] = (df['age'] <= 21).astype(int)  # High growth potential
    df['is_prime'] = ((df['age'] > 21) & (df['age'] <= 29)).astype(int)  # Peak years
    df['is_veteran'] = (df['age'] > 29).astype(int)  # Decline risk
    
    # Rating tier (higher rated players grow slower)
    df['is_elite'] = (df['overall'] >= 85).astype(int)
    df['is_good'] = ((df['overall'] >= 75) & (df['overall'] < 85)).astype(int)
    df['is_average'] = (df['overall'] < 75).astype(int)
    
    # Wage standardization (account for inflation)
    # Z-score = (wage - mean_wage_in_season) / std_wage_in_season
    # This tells us if player is highly paid relative to their contemporaries
    if 'wage_eur' in df.columns and 'season' in df.columns:
        df['wage_eur_numeric'] = pd.to_numeric(df['wage_eur'], errors='coerce').fillna(0)
        df['wage_zscore'] = df.groupby('season')['wage_eur_numeric'].transform(
            lambda x: (x - x.mean()) / x.std() if x.std() > 0 else 0
        )
        # Also add percentile rank (0-100)
        df['wage_percentile'] = df.groupby('season')['wage_eur_numeric'].rank(pct=True) * 100
    
    # Value standardization (same logic)
    if 'value_eur' in df.columns and 'season' in df.columns:
        df['value_eur_numeric'] = pd.to_numeric(df['value_eur'], errors='coerce').fillna(0)
        df['value_zscore'] = df.groupby('season')['value_eur_numeric'].transform(
            lambda x: (x - x.mean()) / x.std() if x.std() > 0 else 0
        )
    
    if not is_gk:
        # Outfield: Expected performance alignment (overperformers grow faster)
        # Goals vs xG shows if player is finishing above/below expectations
        if 'Per 90 Minutes_xG' in df.columns and 'Per 90 Minutes_Gls' in df.columns:
            xG = pd.to_numeric(df['Per 90 Minutes_xG'], errors='coerce').fillna(0)
            actual_goals = pd.to_numeric(df['Per 90 Minutes_Gls'], errors='coerce').fillna(0)
            df['goals_vs_xG'] = actual_goals - xG  # Overperforming = positive
        
        # Position features (FW, MF, DF)
        if 'pos' in df.columns:
            df['pos_lower'] = df['pos'].str.lower().str[:2]  # Get first 2 letters
            df['is_forward'] = (df['pos_lower'] == 'fw').astype(int)
            df['is_midfield'] = (df['pos_lower'] == 'mf').astype(int)
            df['is_defense'] = (df['pos_lower'] == 'df').astype(int)
            df.drop('pos_lower', axis=1, inplace=True)  # Clean up temp column
    
    return df

print("✅ Defined add_engineered_features() function")
print(f"\nWill add these features after splitting datasets:")
print(f"  - Age features: age_squared, is_youth, is_prime, is_veteran")
print(f"  - Rating tiers: is_elite, is_good, is_average")
print(f"  - Wage/Value (inflation-adjusted): wage_zscore, wage_percentile, value_zscore")
print(f"  - Outfield performance: goals_vs_xG")
print(f"  - Outfield positions: is_forward, is_midfield, is_defense")

## 6. Add Lagged Features (Prior Season Stats)

**Key Insight**: We can use ALL 18,907 samples AND add lagged features!

**Strategy**:
1. Create lagged features for all players (season -1 stats)
2. Add `has_prior_season` binary indicator (1 if lag data exists, 0 if not)
3. Fill missing lags with 0
4. Model learns to use lags when available, ignore when not

**Why this works**:
- Players WITH prior data: Model uses current stats + trends
- Players WITHOUT prior data: Model uses only current stats (lags=0, has_prior_season=0)
- No data loss: Keep all 18,907 training samples!

In [ ]:
def create_lagged_features(df):
    """
    Create lagged features (prior season stats) for ALL players.
    Missing lags will be filled with 0 and indicated by has_prior_season=0.
    """
    df = df.copy()
    df = df.sort_values(['player', 'season_year'])
    
    # Key stats to lag (focus on most predictive features)
    lag_features = [
        'overall',           # Rating trajectory
        'age',              # To calculate if they aged
        'Playing Time_Min', # Minutes trend
        'Per 90 Minutes_Gls',   # Goal scoring trend (outfield)
        'Per 90 Minutes_Ast',   # Assist trend (outfield)
        'Per 90 Minutes_G+A',   # G+A trend
        'Per 90 Minutes_xG',    # Expected goals trend
        'value_eur',         # Market value trend
    ]
    
    # Create lag features
    for feature in lag_features:
        if feature in df.columns:
            # Create lag (shift forward by player)
            df[f'{feature}_lag1'] = df.groupby('player')[feature].shift(1)
    
    # Binary indicator: does this player have prior season data?
    df['has_prior_season'] = df['overall_lag1'].notna().astype(int)
    
    # Calculate momentum features from lags (before filling NaN)
    if 'overall_lag1' in df.columns:
        # Rating momentum: Did they grow/decline last season?
        # (current - lag) shows if they're improving
        df['rating_momentum'] = df['overall'] - df['overall_lag1']
    
    if 'Per 90 Minutes_Gls_lag1' in df.columns and 'Per 90 Minutes_Gls' in df.columns:
        # Goals trend: Scoring more or less than last season?
        df['goals_trend'] = df['Per 90 Minutes_Gls'] - df['Per 90 Minutes_Gls_lag1']
    
    if 'Playing Time_Min_lag1' in df.columns:
        # Minutes trend: Playing more or less?
        df['minutes_trend'] = df['Playing Time_Min'] - df['Playing Time_Min_lag1']
    
    # Fill missing lags with 0 (for players without prior season)
    lag_cols = [col for col in df.columns if '_lag1' in col or col in ['rating_momentum', 'goals_trend', 'minutes_trend']]
    df[lag_cols] = df[lag_cols].fillna(0)
    
    print(f"✅ Created lagged features:")
    print(f"   - {len([f for f in lag_features if f in df.columns])} base features lagged")
    print(f"   - Added has_prior_season indicator")
    print(f"   - Added momentum features: rating_momentum, goals_trend, minutes_trend")
    print(f"   - Players with prior season: {df['has_prior_season'].sum()} / {len(df)}")
    
    return df

# Apply to FULL df_model (all 18,907 samples)
df_model = create_lagged_features(df_model)

In [ ]:
# Split datasets (AFTER adding lagged features)
df_gk = df_model[df_model['is_gk']].copy()
df_outfield = df_model[~df_model['is_gk']].copy()

print(f"\n✅ Split into GK ({len(df_gk)}) and Outfield ({len(df_outfield)}) datasets")
print(f"   - GK with prior season data: {df_gk['has_prior_season'].sum()} / {len(df_gk)}")
print(f"   - Outfield with prior season data: {df_outfield['has_prior_season'].sum()} / {len(df_outfield)}")

In [ ]:
# Apply engineered features to both datasets (AFTER splitting)
df_outfield = add_engineered_features(df_outfield, is_gk=False)
df_gk = add_engineered_features(df_gk, is_gk=True)

# Check which features actually exist in the data
print("\n📊 Checking feature availability...")
print(f"\nOutfield features:")
available_outfield = [f for f in outfield_features if f in df_outfield.columns]
missing_outfield = [f for f in outfield_features if f not in df_outfield.columns]
print(f"  Available: {len(available_outfield)}/{len(outfield_features)}")
if missing_outfield:
    print(f"  Missing: {len(missing_outfield)} features (e.g., {missing_outfield[:3]})")
    outfield_features = available_outfield  # Use only available

print(f"\nGK features:")
available_gk = [f for f in gk_features if f in df_gk.columns]
missing_gk = [f for f in gk_features if f not in df_gk.columns]
print(f"  Available: {len(available_gk)}/{len(gk_features)}")
if missing_gk:
    print(f"  Missing: {len(missing_gk)} features: {missing_gk}")
    gk_features = available_gk  # Use only available

print(f"\n✅ Engineered features added to both datasets")

In [ ]:
def prepare_data(df, feature_list):
    """
    Prepare data for XGBoost:
    1. Convert all features to numeric
    2. Handle missing values
    3. Add engineered features to feature list
    """
    # Add engineered features
    engineered = [
        'age_squared', 
        'is_youth', 'is_prime', 'is_veteran', 
        'is_elite', 'is_good', 'is_average',
        'wage_zscore', 'wage_percentile', 'value_zscore'  # Inflation-adjusted features
    ]
    all_features = feature_list + [f for f in engineered if f in df.columns]
    
    # Add outfield-specific engineered features (actual vs expected)
    if 'goals_vs_xG' in df.columns:
        all_features.append('goals_vs_xG')
    
    # Add position features (outfield only)
    position_features = ['is_forward', 'is_midfield', 'is_defense']
    all_features.extend([f for f in position_features if f in df.columns])
    
    # Add lagged features (prior season stats)
    lag_features = [col for col in df.columns if '_lag1' in col or col in ['has_prior_season', 'rating_momentum', 'goals_trend', 'minutes_trend']]
    all_features.extend([f for f in lag_features if f not in all_features])
    
    # Convert all to numeric
    for col in all_features:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    
    # Get feature matrix
    X = df[all_features].copy()
    
    # Replace infinity values with NaN (caused by divisions by zero)
    X = X.replace([np.inf, -np.inf], np.nan)
    
    # Fill missing values with 0 (missing stats usually mean 0 performance)
    X = X.fillna(0)
    
    y = df['rating_change']
    
    return X, y, all_features

# Prepare outfield data
X_outfield, y_outfield, outfield_features_final = prepare_data(df_outfield, outfield_features_simple)

# Prepare GK data
X_gk, y_gk, gk_features_final = prepare_data(df_gk, gk_features_simple)

print(f"Outfield: {X_outfield.shape[0]} samples × {X_outfield.shape[1]} features")
print(f"GK: {X_gk.shape[0]} samples × {X_gk.shape[1]} features")
print(f"\n✅ Data prepared for training")

## 8. Train XGBoost Models

### What is XGBoost?
XGBoost (Extreme Gradient Boosting) is a powerful machine learning algorithm that:
- Builds many small decision trees sequentially
- Each tree learns from the errors of previous trees
- Combines all trees to make final prediction

### Why XGBoost for this problem?
1. **Handles non-linear relationships** (e.g., age curve, diminishing returns)
2. **Feature importance** (tells us what matters most for rating changes)
3. **Robust to missing data**
4. **Fast and accurate**

In [ ]:
def train_xgboost_model(X, y, model_name="Model", custom_params=None):
    """
    Train XGBoost model and evaluate performance
    
    Args:
        X: Feature matrix
        y: Target variable
        model_name: Name for display
        custom_params: Optional dict to override default parameters
    """
    # Split into train/test (80/20)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )
    
    print(f"\n{'='*50}")
    print(f"Training {model_name}")
    print(f"{'='*50}")
    print(f"Train: {len(X_train)} samples")
    print(f"Test: {len(X_test)} samples")
    
    # Default XGBoost parameters (for large datasets)
    params = {
        'objective': 'reg:squarederror',  # Regression task
        'max_depth': 6,                    # Tree depth (prevents overfitting)
        'learning_rate': 0.1,              # How much each tree contributes
        'n_estimators': 200,               # Number of trees
        'min_child_weight': 1,             # Minimum samples required per leaf
        'subsample': 0.8,                  # Use 80% of data per tree
        'colsample_bytree': 0.8,          # Use 80% of features per tree
        'random_state': 42,
    }
    
    # Override with custom parameters if provided
    if custom_params:
        params.update(custom_params)
        print(f"\n🔧 Custom parameters:")
        for key, value in custom_params.items():
            print(f"   {key}: {value}")
    
    # Train model
    model = xgb.XGBRegressor(**params)
    model.fit(X_train, y_train, verbose=False)
    
    # Predictions
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    
    # Evaluation metrics
    train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
    test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
    test_mae = mean_absolute_error(y_test, y_pred_test)
    test_r2 = r2_score(y_test, y_pred_test)
    
    print(f"\n📊 Performance:")
    print(f"  Train RMSE: {train_rmse:.3f}")
    print(f"  Test RMSE:  {test_rmse:.3f}")
    print(f"  Test MAE:   {test_mae:.3f} (avg error in rating points)")
    print(f"  Test R²:    {test_r2:.3f} (0=bad, 1=perfect)")
    
    # Visualize predictions
    plt.figure(figsize=(10, 5))
    
    plt.subplot(1, 2, 1)
    plt.scatter(y_test, y_pred_test, alpha=0.5)
    plt.plot([-5, 5], [-5, 5], 'r--', label='Perfect prediction')
    plt.xlabel('Actual Rating Change')
    plt.ylabel('Predicted Rating Change')
    plt.title(f'{model_name} - Predictions vs Actual')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.subplot(1, 2, 2)
    errors = y_test - y_pred_test
    plt.hist(errors, bins=30, edgecolor='black')
    plt.xlabel('Prediction Error (Actual - Predicted)')
    plt.ylabel('Count')
    plt.title(f'{model_name} - Error Distribution')
    plt.axvline(0, color='red', linestyle='--')
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    return model, X_test, y_test, y_pred_test

# Train both models with different parameters


# GK model: Smaller dataset (1,213 samples), needs simpler model to avoid overfitting
# print("\n🎯 Training Goalkeeper Model (1,213 samples)...")
# print("   Using reduced complexity to prevent overfitting...")
# model_gk, X_test_gk, y_test_gk, y_pred_gk = train_xgboost_model(
#     X_gk, y_gk, "Goalkeeper Model", 
#     custom_params={
#         'max_depth': 3,           # Reduced from 6 → shallower trees
#         'n_estimators': 300,      # Reduced from 200 → fewer trees
#         'min_child_weight': 3,    # Increased from 1 → need more samples per leaf
#         'learning_rate': 0.05,    # Reduced from 0.1 → slower, more careful learning
#     }
# )

## 8b. EXPERIMENT: Train with Simplified Features

Re-run training with the simplified core feature set to compare performance.

In [ ]:
# Prepare data with SIMPLIFIED features
X_outfield_simple, y_outfield_simple, outfield_features_simple_final = prepare_data(df_outfield, outfield_features_simple)
X_gk_simple, y_gk_simple, gk_features_simple_final = prepare_data(df_gk, gk_features_simple)

print(f"📊 Simplified Dataset:")
print(f"Outfield: {X_outfield_simple.shape[0]} samples × {X_outfield_simple.shape[1]} features")
print(f"GK: {X_gk_simple.shape[0]} samples × {X_gk_simple.shape[1]} features")
print(f"\n🔥 Feature reduction:")
print(f"   Outfield: {X_outfield.shape[1]} → {X_outfield_simple.shape[1]} ({X_outfield.shape[1] - X_outfield_simple.shape[1]} fewer)")
print(f"\n✅ Ready to train simplified models")

In [ ]:
# Save FULL engineered dataframes (with all columns including target)
df_outfield.to_csv('../data/clean/outfield_engineered.csv', index=False)
df_gk.to_csv('../data/clean/gk_engineered.csv', index=False)
#add_engineered_features(df_model).to_csv('../data/clean/full_model_engineered.csv', index=False)

print("✅ Full engineered data saved")

# Save PREDICTOR-ONLY data (X) for clean training
# These have ONLY the features needed for prediction, no target variable
X_outfield_simple.to_csv('../data/clean/X_outfield.csv', index=False)
X_gk_simple.to_csv('../data/clean/X_gk.csv', index=False)

print("✅ Predictor data (X) saved:")
print(f"   - X_outfield.csv: {X_outfield_simple.shape[0]} rows × {X_outfield_simple.shape[1]} features")
print(f"   - X_gk.csv: {X_gk_simple.shape[0]} rows × {X_gk_simple.shape[1]} features")

# Save target variables (y) separately
y_outfield_simple.to_csv('../data/clean/y_outfield.csv', index=False, header=['rating_change'])
y_gk_simple.to_csv('../data/clean/y_gk.csv', index=False, header=['rating_change'])

print("✅ Target data (y) saved:")
print(f"   - y_outfield.csv: {len(y_outfield_simple)} values")
print(f"   - y_gk.csv: {len(y_gk_simple)} values")

print("\n💡 In cleanTraining.ipynb, you can now just load:")
print("   X = pd.read_csv('../data/clean/X_outfield.csv')")
print("   y = pd.read_csv('../data/clean/y_outfield.csv')['rating_change']")

#df_outfield

In [ ]:
# Train SIMPLIFIED models

# Outfield with simplified features
print("\n🎯 Training SIMPLIFIED Outfield Model...")
print(f"   Using {X_outfield_simple.shape[1]} features (vs {X_outfield.shape[1]} full)")
model_outfield_simple, X_test_out_s, y_test_out_s, y_pred_out_s = train_xgboost_model(
    X_outfield_simple, y_outfield_simple, "Outfield Model (Simplified)",
    custom_params={'n_estimators': 500,
                   'max_depth': 6,
                   'learning_rate': .05}  # Slightly fewer trees for simpler features
)

# GK with simplified features (same as full since GK already lean)
print("\n🎯 Training SIMPLIFIED GK Model...")
model_gk_simple, X_test_gk_s, y_test_gk_s, y_pred_gk_s = train_xgboost_model(
    X_gk_simple, y_gk_simple, "GK Model (Simplified)", 
    custom_params={
        'max_depth': 3,
        'n_estimators': 200,
        'min_child_weight': 3,
        'learning_rate': 0.05,
    }
)

print("\n" + "="*60)
print("📊 COMPARISON: Full vs Simplified")
print("="*60)
print(f"\nOutfield Model:")
print(f"  Full ({X_outfield.shape[1]} features):       Test RMSE likely ~2.09")
print(f"  Simplified ({X_outfield_simple.shape[1]} features): Test RMSE shown above")
print(f"\n💡 Better generalization? Check if Test RMSE improved with fewer features!")

## 9. Feature Importance

Understand which features matter most for rating changes.

### 📊 Understanding Model Performance Metrics

Before we look at feature importance, let's understand what those numbers mean:

**RMSE (Root Mean Squared Error)**: 
- Average error in rating points
- Lower = better
- Example: RMSE of 1.5 means predictions are off by ~1.5 rating points on average
- **Good**: < 2.0, **Okay**: 2.0-3.0, **Bad**: > 3.0

**MAE (Mean Absolute Error)**:
- Similar to RMSE but easier to interpret
- "On average, how many rating points are we wrong by?"

**R² Score (R-squared)**:
- Measures how well the model explains the data
- Range: 0 to 1 (can be negative if terrible)
- **0.0** = Model is useless (random guessing)
- **0.5** = Model explains 50% of rating changes
- **1.0** = Perfect predictions
- **Good**: > 0.4, **Great**: > 0.6

### 🤔 Questions to Ask Yourself:

1. Is the train RMSE much better than test RMSE?
   - If yes → **Overfitting** (model memorized training data)
   - Solution: Decrease `max_depth` or increase `min_child_weight`

2. Is the R² score low (< 0.3)?
   - Model might need more/better features
   - Or the problem is inherently random (FIFA ratings are subjective!)

3. Are predictions clustered around 0 (no change)?
   - Model is playing it safe
   - Might need to adjust `scale_pos_weight` or collect more extreme examples

In [ ]:
def plot_feature_importance(model, feature_names, model_name, top_n=15):
    """
    Plot top N most important features
    """
    importance = model.feature_importances_
    indices = np.argsort(importance)[-top_n:]
    
    plt.figure(figsize=(10, 6))
    plt.barh(range(len(indices)), importance[indices])
    plt.yticks(range(len(indices)), [feature_names[i] for i in indices])
    plt.xlabel('Feature Importance')
    plt.title(f'{model_name} - Top {top_n} Most Important Features')
    plt.tight_layout()
    plt.show()
    
    # Print top features
    print(f"\n🔝 Top {top_n} features for {model_name}:")
    for i, idx in enumerate(indices[::-1]):
        print(f"{i+1}. {feature_names[idx]}: {importance[idx]:.4f}")

# Plot importance for both models
plot_feature_importance(model_outfield_simple, outfield_features_simple_final, "Outfield Model")
plot_feature_importance(model_gk_simple, gk_features_simple_final, "GK Model")

### 🔍 What is Feature Importance?

Feature importance tells us **which stats matter most** for predicting rating changes.

**How to interpret**:
- Higher number = more important
- Numbers are relative (sum to 1.0)

**Example**: If `age` has importance 0.20 and `goals_90` has 0.10:
- Age is 2x more important than goals/90
- Age contributes 20% to predictions

**Why this matters**:
- Focus on tracking important stats
- Understand what EA Sports values
- Build better player development strategies

**Expected patterns**:
- `age` should be very important (youth grow, veterans decline)
- `overall` (current rating) matters (lower rated = more room to grow)
- Performance stats (goals, assists) should be important for attackers

## 10. Save Models

In [ ]:
import pickle

# Save models and feature lists
models_dir = '../model'
import os
os.makedirs(models_dir, exist_ok=True)

# Save outfield model
with open(f'{models_dir}/model_outfield.pkl', 'wb') as f:
    pickle.dump({
        'model': model_outfield_simple,
        'features': outfield_features_simple_final
    }, f)

# Save GK model
with open(f'{models_dir}/model_gk.pkl', 'wb') as f:
    pickle.dump({
        'model': model_gk_simple,
        'features': gk_features_simple_final
    }, f)

print("✅ Models saved to model/ directory")
print(f"  - model_outfield.pkl ({len(outfield_features_simple_final)} features)")
print(f"  - model_gk.pkl ({len(gk_features_simple_final)} features)")

## 10b. Multi-Target Prediction: Stats Forecasting

Instead of just predicting rating changes, let's predict **individual performance stats** for next season:

**Outfield Targets:**
- Goals per 90
- Assists per 90
- Passing completion %
- Tackles per 90
- FIFA Face Stats: pace, shooting, passing, dribbling, defending, physic

**GK Targets:**
- Saves per 90
- Save %
- FIFA GK Stats: diving, handling, kicking, positioning, reflexes

This gives us detailed player projections beyond just overall rating!

## 15. Summary

### 🎉 What We Built:

1. ✅ **Data Pipeline**: Processed 18,907 player-seasons with position-specific features
2. ✅ **Rating Prediction Models**: Separate XGBoost models for outfield (R² 0.509) and goalkeepers
3. ✅ **Multi-Stat Prediction**: 10+ individual stat models per position type
4. ✅ **Feature Engineering**: Age curves, career stages, performance metrics, lagged features
5. ✅ **Auto-Scout**: Comprehensive next-season prediction with all stats
6. ✅ **Model Bundles**: Packaged rating + stat models for easy deployment

### 📊 Prediction Coverage:

**Outfield Players:**
- ✅ Overall Rating
- ✅ Goals per 90
- ✅ Assists per 90
- ✅ Pass Completion %
- ✅ Tackles per 90
- ✅ FIFA Face Stats: Pace, Shooting, Passing, Dribbling, Defending, Physical

**Goalkeepers:**
- ✅ Overall Rating
- ✅ Saves per 90
- ✅ Save %
- ✅ GK Attributes: Diving, Handling, Kicking, Positioning, Reflexes

### 🚀 Model Files Saved:

- `model_outfield_bundle.pkl` - Rating + 10 stat models
- `model_gk_bundle.pkl` - Rating + 7 stat models
- Legacy files: `model_outfield.pkl`, `model_gk.pkl` (rating only)

### 📈 Next Actions:

1. ✅ Run the training cells above to generate all models
2. Test predictions with various players and seasons
3. Integrate bundles into FastAPI backend
4. Build frontend UI:
   - Player search interface
   - Current vs predicted stats comparison
   - FIFA card visualization with predicted attributes
   - Interactive stat explorer
5. Add confidence intervals (optional enhancement)
6. Create batch prediction endpoint for multiple players

### 💡 Key Insights:

- Rating predictions are accurate within ~1.5 points on average
- Age is the strongest predictor (youth grow, veterans decline)
- FIFA attributes capture EA's evaluation philosophy
- Lagged features (prior season trends) significantly improve accuracy
- Individual stat predictions enable detailed player development tracking